In [1]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
# import StringIO

def list_request(page_no, keyword=''):
    url="http://api.epeople.go.kr/soap/CivilAppealPolicyQnAService?wsdl"
    headers = {'content-type': 'text/xml; charset=UTF-8'}
    body = """
    <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:ns3="http://apache.org/headers" xmlns:ns2="http://soap.qna.pc.acrc/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:nil="true">
       <soap:Header>
       </soap:Header>
       <soap:Body>
          <ns2:getCivilAppealPolicyQnaList>
             <QnaListRequest>
                <type>1</type>
                <nowPageNo>{0}</nowPageNo>
                <keyword>{1}</keyword>
                <pageMg>10</pageMg>
             </QnaListRequest>
          </ns2:getCivilAppealPolicyQnaList>
       </soap:Body>
    </soap:Envelope>
    """.format(page_no, keyword).encode('utf8')
    response = requests.post(url,data=body,headers=headers)
    content = response.content.decode('utf8')
    root = etree.fromstring(content)
    body = root.getchildren()[0]
    resp = body.find('{http://soap.qna.pc.acrc/}getCivilAppealPolicyQnaListResponse')
    resp = resp.find('QnaListResponse')
    infos = resp.findall('QnaListInfo')
    if not infos:
        return None
    result = []
    for child in infos:
        result.append([var.text for var in child.getchildren()])
    return result

def item_request(faq_no, peti_no, civil_no):
    url="http://api.epeople.go.kr/soap/CivilAppealPolicyQnAService?wsdl"
    headers = {'content-type': 'text/xml; charset=UTF-8'}
    body = """
    <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:ns3="http://apache.org/headers" xmlns:ns2="http://soap.qna.pc.acrc/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:nil="true">
       <soap:Header>

       </soap:Header>
       <soap:Body>
          <ns2:getCivilAppealPolicyQnaItem>
             <QnaItemRequest>
                <faqNo>{0}</faqNo>
                <petiNo>{1}</petiNo>
                <civilNo>{2}</civilNo>
                <type>3</type>
             </QnaItemRequest>
          </ns2:getCivilAppealPolicyQnaItem>
       </soap:Body>
    </soap:Envelope>
    """.format(faq_no, peti_no, civil_no).encode('utf8')
    response = requests.post(url,data=body,headers=headers)
    content = response.content.decode('utf8')
    root = etree.fromstring(content)
    body = root.getchildren()[0]
    resp = body.find('{http://soap.qna.pc.acrc/}getCivilAppealPolicyQnaItemResponse')
    infos = resp.find('QnaItemResponse')
    anc = infos.find('ancName').text
    question = infos.find('content').text
    soup = BeautifulSoup(infos.find('ansContent').text, 'html.parser')
    answer = soup.find_all('span')
    answer = ''.join([x.text for x in answer])
    return anc, question, answer

In [245]:
li = list_request(9000)
print(li)

[['국토교통부', '1613000', '338705', '1611000008', '2016-09-30 12:55:13', '하천점용허가의 유효기간', '2'], ['국토교통부', '1613000', '141124', '1611000008', '2005-04-18 00:00:00', '하천점용허가 권리·의무 승계 신고', '2'], ['국토교통부', '1613000', '144296', '1611000008', '2007-02-04 18:30:10', '하천에서 골재 채취시 토지점용료 납부 여부', '2'], ['국토교통부', '1613000', '144527', '1611000008', '2007-04-20 11:48:39', '하천점용허가 가능 여부에 대하여 사전 확인이 가능한지요?', '2'], ['국토교통부', '1613000', '338711', '1611000008', '2016-09-30 15:21:51', '하천점용 허가신청시 인근 토지소유자의 동의', '2'], ['국토교통부', '1613000', '338712', '1611000008', '2016-09-30 15:39:51', '하천에서 &lt;지구&gt;의 의미', '2'], ['국토교통부', '1613000', '338715', '1611000008', '2016-09-30 16:45:44', '하천점용허가 기관', '2'], ['국토교통부', '1613000', '144471', '1611000008', '2007-03-22 16:27:42', '하천점용허가시 납부하여야 하는 비용', '2'], ['공정거래위원회', '1130000', '451964', '10', '2018-10-15 14:51:11', '태권도장에 학교 명칭 사용 가부', '2'], ['경찰청', '1320000', '451344', '0060020000', '2018-10-11 10:18:16', '사은품 편취 사기신고', '2']]


In [246]:
anc, question, answer = item_request(552450, 'A000001570', '3130000')
print(anc, question, answer)

서울특별시 마포구 장애인 복지카드 신청 방법 및 처리 안내 요청합니다. ○ 장애인복지카드란?장애인에게 발급되는 장애인등록증 및 신용카드나 직불카드와 통합된 등록증을 발급​○ 지원대상​장애인복지법 제32조의 2에 의한 동록 장애인에게는 본인의 희망에 따라 장애인등록증 또는 장애인복지카드중 1종류만을 발급함​○ 장애인복지카드 발급 안내​장애인등록증 등은 (장애인등록증 개선사업을위한 협약)에 의하여 보건복지부,신한카드사, 한국조폐공사간의 협약에 의해서 제작관리○ 온라인 신청 가능한 경우온라인 신청시 장애인 본인이 신청(장애인으로 등록된 장애인 본인만 신청가능​)* 단 복지카드를 신용카드 또는 직불카드 형태로 신청하는 경우 19세 미만의 장애인은 발급대상에서 제외됨​○ 온라인 신청이 불가능한 경우​*신청인이 공인인증서가 없는 경우*신청인이 외국국적인 경우○ 장애인복지카드 신청방법초기상담및 서비스 신청(읍,면,동 주민센터에 방문신청및 온라인 신청) → 접수(읍,면,동 주민센터에서 접수 처리) → 제작(조폐공사) → 교부(읍,면,동 주민센터에서 교부하거나 동기수행)


In [7]:
import csv
idx = 1
csvfile = open('data.csv', 'w', encoding='utf-8')
csvwriter = csv.writer(csvfile)
while True:
    li = list_request(idx)
    for item in li:
        faq_no, peti_no, civil_no = item[2], item[3], item[1]
        anc, question, answer = item_request(faq_no, peti_no, civil_no)
        if  answer:
            csvwriter.writerow([anc, question, answer])
    if idx == 10:
        break
    idx += 1
    

In [8]:
with open('data.csv', newline='', encoding='utf-8') as csvfile:
    data_reader = csv.reader(csvfile)
    csv_data = [row for row in data_reader]
csv_data

[['서울특별시 마포구',
  '서울형 주택바우처 신청 자격이 어떻게 되나요?',
  '서울형 주택바우처 신청 자격은 다음과 같습니다.1. 신청대상(아래 기준 모두 충족)o 민간 (보증부)월세 주택 및 고시원 거주자o 전세전환가액이 9,500만원 이하인 가구※ 전세전환가액 = (월 임대료 x 75) + 임대보증금가구원수1인2인3인4인5인중위소득 60%1,024,2051,743,9172,256,0192,768,1223,280,224o 소득인정액이 기준 중위소득의 60% 이하인 가구2. 제외대상o 생계급여 수급자, 의료급여 수급자, 주거급여 수급자o 공공임대주택(국민임대주택, 장기전세주택, 재개발임대주택, 행복주택, 영구임대주택, 전세임대주택, 매입임대주택) 거주자o 가구원이 대학교 재학·휴학생 등 학생으로만 구성된 가구o 외국인으로만 구성된 가구o 가구원이 자동차 2대 이상 소유한 경우(차량종류 불문)o 준주택(기숙사, 오피스텔, 노인복지주택) 거주 가구o 근린생활시설(고시원 제외) 거주 가구o 전세 계약 가구o 전대차(임차인으로부터 재임차) 계약 가구o 부양의무자와 임대차 계약 가구'],
 ['서울특별시 마포구',
  '청년희망키움통장을 신청하고 싶은데 신청방법 및 처리절차는 어떻게 되나요?',
  '청년희망키움통장 신청방법 및 처리절차는 다음과 같습니다.1. 신청대상(아래 기준 모두 충족)o 생계급여 수급자 또는 의료급여 수급자 o 만 15세 ~ 만 34세 청년 o 근로·사업소득이 기준 중위소득의 20% 이상 가구※ 2019년 기준 중위소득 20% : 341,402원2. 신청방법 및 처리절차 o 신청 장소 : 관할 주민센터 방문 신청o 신청 서류 : 희망키움통장 참여(변경) 신청서, 저축동의서, 개인정보 제공 및 활용 동의서, 소득증빙서류o 조사 실시 및 결과 통보 : 접수일로부터 15일 이내 처리o 통장 개설 : 가까운 하나은행 지점을 통해 통장 개설※ 비대면 계좌(인터넷뱅킹 및 스마트폰뱅킹) 개설 가능3. 제외대상o 주거급여 수급자, 교육급여 수급자, 차